In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, date
from IPython.display import display, Markdown
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

In [ ]:
df = pd.read_csv("/Users/martina.megasari/workspace/wids_datathon/data/widsdatathon2023/train_data.csv")
df_test = pd.read_csv("/Users/martina.megasari/workspace/wids_datathon/data/widsdatathon2023/test_data.csv")
print(f"df shape:{df.shape}")
print(f"df_test shape:{df_test.shape}")

In [ ]:
df.head()

In [ ]:
df["startdate"] = pd.to_datetime(df["startdate"])
df["startdate"].head(5)

# Missing values

In [ ]:
df_missing = df.isna().sum(axis=0)
df_missing[df_missing > 0]

In [ ]:
cols_missing_values = [
    "nmme0-tmp2m-34w__ccsm30",
    "nmme-tmp2m-56w__ccsm3",
    "nmme-prate-34w__ccsm3",
    "nmme0-prate-56w__ccsm30",
    "nmme0-prate-34w__ccsm30",
    "nmme-prate-56w__ccsm3",
    "nmme-tmp2m-34w__ccsm3",
    "ccsm30"]
cols_missing_values_with_date = cols_missing_values.copy()
cols_missing_values_with_date.append("startdate")
df.loc[df[cols_missing_values].isna().any(axis=1), cols_missing_values_with_date]

There seem to be some pattern in the missing values. When nmme-tmp2m-56w__ccsm3 is NaN, nmme0-prate-56w__ccsm30, nmme-prate-56w__ccsm3, ccsm30 are also NaN

In [ ]:
df["month"] = df["startdate"].dt.month
df["year"] = df["startdate"].dt.year
for col in cols_missing_values:
    display(
        df.loc[df[col].isna(), [col,"year","month"]].drop_duplicates()
    )

In [ ]:
# Regardless of the pattern, 2016 doesn't have any missing values. 
# So it's very likely just some missing measurements. 
# Let's follow the community to use forward fill.
df = df.sort_values(["lat","lon","startdate"]).ffill()

In [ ]:
# remove useless index
df = df.drop("index", axis=1)

# EDA

In [ ]:
corr_coeff = df.corr(numeric_only=True)

In [ ]:
plt.figure(figsize=(15,15))
ax = sns.heatmap(corr_coeff)
plt.show()

In [ ]:
corr_coeff.loc[target].sort_values()

# Feature engineering

In [ ]:
# constant
current_date = date.today()
current_year = current_date.year

In [ ]:
def prepare(df):
    # constant
    current_date = date.today()
    current_year = current_date.year
    
    df["startdate"] = pd.to_datetime(df["startdate"], format="%m/%d/%y")
    df["year"] = df["startdate"].dt.year - current_year
    df["month"] = df["startdate"].dt.month
    df["day"] = df["startdate"].dt.day
    df["month_sin"] = np.sin(2*np.pi*df["month"]/12)
    df["month_cos"] = np.cos(2*np.pi*df["month"]/12)
    df["day_sin"] = np.sin(2*np.pi*df["startdate"].dt.dayofyear/365)
    df["day_cos"] = np.cos(2*np.pi*df["startdate"].dt.dayofyear/365)

    df_climate = pd.get_dummies(df["climateregions__climateregion"], drop_first=True, prefix="climate_region")
    df = pd.concat([df, df_climate], axis=1)
    return df

In [ ]:
df = prepare(df)

# Model development

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [ ]:
target="contest-tmp2m-14d__tmp2m"

In [ ]:
def train_cv(df, features, target, params, num_boost_round=100, n_folds=5):
    random_state = 13
    folds = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    scores = []
    for idx, (train_idx, val_idx) in enumerate(folds.split(df[features], df[target])):
    #     print(train_idx)
        X_train = df.loc[train_idx, features]
        y_train = df.loc[train_idx, target]
        train_data = lgb.Dataset(X_train, label=y_train)

        X_val = df.loc[val_idx, features]
        y_val = df.loc[val_idx, target]
        val_data = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(params=params, train_set=train_data, num_boost_round=num_boost_round, valid_sets=[train_data, val_data])
        y_val_pred = model.predict(X_val)

        scores.append(mean_squared_error(y_val_pred, y_val))
    return scores

def create_submission(df_test, fn_prepare, model, features, target, filename):
    df_test = fn_prepare(df_test)
    df_test[target] = model.predict(df_test[features])
    df_test[[target,"index"]].to_csv("submission.csv", index=False)

In [ ]:
non_features = ["index","startdate","year","month","day", "contest-tmp2m-14d__tmp2m", "climateregions__climateregion"]
features = [col for col in df.columns if col not in non_features ]
features

## Experiment 1 - all features: 1.392
Using all the features with one hot encoded climate regions.
Observation: 
- overfitting
- need to fix the cross validation?

In [ ]:
params = {
    "boosting_type": "gbdt",
    "objective": "regression_l2",
    "metric": "mean_squared_error",
    "num_leaves": 100,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": 2
}
scores = train_cv(df, features, target, params, num_boost_round=100, n_folds=5)

In [ ]:
X = df[features]
y = df[target]
train_data = lgb.Dataset(X, label=y)
final_model = lgb.train(params=params, train_set=train_data, num_boost_round=200, valid_sets=[train_data])

In [ ]:
create_submission(df_test, prepare, final_model, features, target, "submission1.csv")

In [ ]:
df_feature_importance = pd.DataFrame({
    "feature": features,
    "feature_importance": final_model.feature_importance()
})
features_top15 = df_feature_importance.sort_values("feature_importance", ascending=False).head(15)["feature"].values
features_top10 = df_feature_importance.sort_values("feature_importance", ascending=False).head(10)["feature"].values
features_top7 = df_feature_importance.sort_values("feature_importance", ascending=False).head(7)["feature"].values

## Experiment 2 - top 10 features: 1.118
We haven't fixed the cross validation, but let's try to use only the top10 features from the model from experiment 1.<br>
Observation: 
- Much better score
- CV score is much closer to the test score

In [ ]:
params = {
    "boosting_type": "gbdt",
    "objective": "regression_l2",
    "metric": "mean_squared_error",
    "num_leaves": 100,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": 1
}
scores_features_top10 = train_cv(df, features_top10, target, params)

In [ ]:
X = df[features_top10]
y = df[target]
train_data = lgb.Dataset(X, label=y)
model_features_top10 = lgb.train(params=params, train_set=train_data, num_boost_round=200, valid_sets=[train_data])

In [ ]:
create_submission(df_test, prepare, model_features_top10, features_top10, target, "submission_features_top10.csv")

## Experiment 3 - top 7 features: 1.455
Top 10 features gave quite a big improvement, let's try top 7 features

In [ ]:
X = df[features_top7]
y = df[target]
train_data = lgb.Dataset(X, label=y)
model_features_top7 = lgb.train(params=params, train_set=train_data, num_boost_round=200, valid_sets=[train_data])

In [ ]:
create_submission(df_test, prepare, model_features_top7, features_top7, target, "submission_features_top7.csv")

# Experiment 4 - top 15 features: 1.1.52


In [ ]:
X = df[features_top15]
y = df[target]
train_data = lgb.Dataset(X, label=y)
model_features_top15 = lgb.train(params=params, train_set=train_data, num_boost_round=200, valid_sets=[train_data])

In [ ]:
create_submission(df_test, prepare, model_features_top15, features_top15, target, "submission_features_top15.csv")

# Experiment 5 - PCA(n_components=5): not submitted

In [ ]:
from sklearn.decomposition import PCA
pca_weather = PCA(n_components=50)
pc_weather = pca_weather.fit_transform(df[features])

In [ ]:
df_test_prepared = prepare(df_test)
df_test_pc_weather = pca_weather.transform(df_test_prepared[features])

In [ ]:
y = df[target]
train_data = lgb.Dataset(pc_weather, label=y)
model_pca5 = lgb.train(params=params, train_set=train_data, num_boost_round=200, valid_sets=[train_data])
df_test[target] = model_pca5.predict(df_test_pc_weather)